In [1]:
import io 
import pandas as pd
trainG = pd.read_excel('../input/train-test-sentiment-analysis/trainG_clean.xlsx')
test = pd.read_excel('../input/train-test-sentiment-analysis/test_clean.xlsx') 

In [2]:
trainG

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl frie...
1,2,0,i missed the new moon trail...
2,3,1,omg its already o
3,4,0,omgaga im sooo im gunna cry i'...
4,5,0,i think mi bf is cheating on me ...
...,...,...,...
75175,99995,0,cupati it took me waaay too long to get your ...
75176,99996,0,cupcake seems like a repeating problem hop...
75177,99998,0,cupcake ya i thought so
75178,99999,1,cupcake dollie yes yes i'm glad you had mor...


In [3]:
test = test[test['ItemID'].map(len) < 10]
test

,ItemID,SentimentText
0,1,is so sad for my APL frien...
1,2,httI missed the New Moon tr...
2,3,omg its already 7:30 :O;;;;;;;;;...
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...
...,...,...
299037,299993,@MartinDubow Welcome to my friends list and th...
299038,299994,@MartinDubow yeah and I envy u. ;;;;;;;;;;;;;;...
299042,299998,@MartinezMayra I got u at work when I go buy o...
299043,299999,@martinfaux He certainly gives good interview ...


In [4]:
data = trainG.copy()
data.drop('ItemID', axis=1, inplace=True)
print(data.isnull().any())
data['Sentiment'] = data['Sentiment'].astype('category')
print(type(data['Sentiment'][0]))
data['label_id'] = data['Sentiment'].cat.codes
data['label_id'].head()

Sentiment        False
SentimentText    False
dtype: bool
<class 'numpy.int64'>


0    0
1    0
2    1
3    0
4    0
Name: label_id, dtype: int8

In [5]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

data['SentimentText']  = data['SentimentText'].astype(str)
test['SentimentText'] = test['SentimentText'].astype(str)
data['SentimentText'] = preprocess_reviews(data['SentimentText'])

test['SentimentText'] = preprocess_reviews(test['SentimentText'])

In [6]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score
import six.moves.cPickle as pickle


In [7]:

import numpy as np 
import pandas as pd

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [8]:
nlp=spacy.load("en_core_web_sm")

In [9]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
import string
punct=string.punctuation

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [10]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [11]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [12]:
x = trainG['SentimentText']
y = trainG['Sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [14]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [15]:
clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f610374acb0>)),
                ('clf', LinearSVC())])

In [16]:
y_pred = clf.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.67      0.69      6692
           1       0.75      0.77      0.76      8344

    accuracy                           0.73     15036
   macro avg       0.73      0.72      0.72     15036
weighted avg       0.73      0.73      0.73     15036



In [ ]:
y_pred=clf.predict(test['SentimentText'])

In [ ]:
y_pred

In [ ]:
y_pred[1]

In [ ]:
sub_file=pd.DataFrame({'id':test['ItemID'],'target':y_pred.round().astype(int)})

In [ ]:
prediction = list() 
for i in range(len(test['SentimentText'])):
  prediction.append(y_pred[i]) 


test['prediction'] = prediction  

In [ ]:


test.to_excel('resultat_SVM.xlsx' , index=False)


 Another Model : Naive bayes 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from multiprocessing import Pool

In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                       ngram_range=(1,3),
                       max_df=1.0,
                       min_df=10,
                       max_features=500,
                       norm='l2',
                       sublinear_tf=True)

In [ ]:
train_features = tfidf.fit_transform(X_train).toarray()
print(train_features.shape)

In [ ]:
test_features = tfidf.transform(X_test).toarray()
print(test_features.shape)

In [ ]:
train_labels = y_train
test_labels = y_test

In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
mnb_classifier = MultinomialNB()

In [ ]:
mnb_classifier.fit(train_features, train_labels)

In [ ]:
mnb_prediction = mnb_classifier.predict(test_features)

In [ ]:
training_accuracy = accuracy_score(train_labels, mnb_classifier.predict(train_features))
print(training_accuracy)

In [ ]:
testing_accuracy = accuracy_score(test_labels, mnb_prediction)
print(testing_accuracy)

In [ ]:
print(classification_report(test_labels, mnb_prediction))

In [ ]:
conf_matrix = confusion_matrix(test_labels, mnb_prediction)
print(conf_matrix)

In [ ]:
test_vectorizer =tfidf.transform(test['SentimentText']).toarray()

In [ ]:
test_vectorizer.shape

In [ ]:
final_predictions = mnb_classifier.predict(test_vectorizer)

In [ ]:
submission_df = pd.DataFrame()

In [ ]:
submission_df['Id'] = test['ItemID']
submission_df['target'] = final_predictions

In [ ]:
submission_df['target'].value_counts()

In [ ]:
submission = submission_df.to_csv('resultat_NB.csv',index = False)
